<a href="https://colab.research.google.com/github/marcogemaque/COVID-19-Breathers/blob/main/Mechanical_Ventilator_COVID_19_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**A Duck's Dream**

### **Breathers on COVID-19**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_real = pd.read_csv("/content/drive/My Drive/A Duck's Dream/distribuicao_respiradores.csv", sep=';')

In [ ]:
df = df_real.copy()
df.head()

,DATA,FORNECEDOR,DESTINO,ESTADO/MUNICIPIO,TIPO,QUANTIDADE,VALOR,DESTINATARIO,UF,DATA DE ENTREGA
0,19/04/2020,VYAIRE,RIO DE JANEIRO,ESTADO,UTI,40,"2400000,00",ALMOXARIFADO ESTADUAL DE MEDICAMENTOS - RIO DE...,RJ,19/04/2020
1,22/04/2020,VYAIRE,RIO DE JANEIRO,ESTADO,UTI,20,"1200000,00",LABORATÓRIO QUÍMICO FARMACÊUTICO DE AERONÁUTICA,RJ,23/04/2020
2,08/05/2020,MAGNAMED,RIO DE JANEIRO,ESTADO,UTI,25,"1500000,00",ALMOXARIFADO ESTADUAL DE MEDICAMENTOS - RIO DE...,RJ,09/05/2020
3,08/05/2020,KTK,RIO DE JANEIRO,ESTADO,UTI,15,"900000,00",ALMOXARIFADO ESTADUAL DE MEDICAMENTOS - RIO DE...,RJ,09/05/2020
4,09/05/2020,LEISTUNG,RIO DE JANEIRO,ESTADO,UTI,6,"360000,00",ALMOXARIFADO ESTADUAL DE MEDICAMENTOS - RIO DE...,RJ,09/05/2020


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1647 entries, 0 to 1646
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   DATA              1647 non-null   object
 1   FORNECEDOR        1647 non-null   object
 2   DESTINO           1647 non-null   object
 3   ESTADO/MUNICIPIO  1647 non-null   object
 4   TIPO              1647 non-null   object
 5   QUANTIDADE        1647 non-null   int64 
 6   VALOR             1647 non-null   object
 7   DESTINATARIO      1647 non-null   object
 8   UF                1647 non-null   object
 9   DATA DE ENTREGA   1647 non-null   object
dtypes: int64(1), object(9)
memory usage: 128.8+ KB


In [ ]:
df['VALOR'] = df['VALOR'].str.replace(',','.')
df['VALOR'] = pd.to_numeric(df['VALOR'], errors='coerce')
df.tail()

,DATA,FORNECEDOR,DESTINO,ESTADO/MUNICIPIO,TIPO,QUANTIDADE,VALOR,DESTINATARIO,UF,DATA DE ENTREGA
1642,24/09/2020,VYAIRE LTV 2200 USA,PARANA,MUNICIPIO,TRANSPORTE USA,1,63125.97,SMS de GUAIRA,PR,28/09/2020
1643,08/10/2020,VYAIRE LTV 2200 USA,PARANA,ESTADO,TRANSPORTE USA,20,1262519.40,5o Batalhao de Suprimento,PR,17/10/2020
1644,08/10/2020,MAGNAMED,PARANA,ESTADO,UTI,4,240000.00,5o Batalhao de Suprimento,PR,17/10/2020
1645,16/10/2020,MAGNAMED,PARANA,MUNICIPIO,UTI,1,60000.00,SMS de WENCESLAU BRAZ,PR,21/10/2020
1646,07/08/2020,MAGNAMED,LIBANO,-,TRANSPORTE,300,14487000.00,MISSÃO FAB (Doação Destino ao Libano).,-,08/08/2020


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1647 entries, 0 to 1646
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DATA              1647 non-null   object 
 1   FORNECEDOR        1647 non-null   object 
 2   DESTINO           1647 non-null   object 
 3   ESTADO/MUNICIPIO  1647 non-null   object 
 4   TIPO              1647 non-null   object 
 5   QUANTIDADE        1647 non-null   int64  
 6   VALOR             1647 non-null   float64
 7   DESTINATARIO      1647 non-null   object 
 8   UF                1647 non-null   object 
 9   DATA DE ENTREGA   1647 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 128.8+ KB


In [ ]:
df_date = df['DATA'].str.contains('[0-9]{2}.[0-9]{2}.[0-9]{4}')
df_date = df_date.loc[df_date == False]
df_date

300     False
837     False
1503    False
Name: DATA, dtype: bool

In [ ]:
sel = df.iloc[[300, 837, 1503]]
sel

,DATA,FORNECEDOR,DESTINO,ESTADO/MUNICIPIO,TIPO,QUANTIDADE,VALOR,DESTINATARIO,UF,DATA DE ENTREGA
300,ESTADO,WEG,DISTRITO FEDERAL,ESTADO,UTI,25,1500000.0,Secretaria Estadual de Saúde,DF,10/06/2020
837,2020-04-30 00:00:00,LEISTUNG,PARA,ESTADO,UTI,60,3600000.0,Secretaria Estadual de Saúde,PA,31/04/2020
1503,16/10/M2020,MAGNAMED,RIO GRANDE DO SUL,MUNICIPIO,UTI,1,60000.0,SECR MUN DE SAUDE DE MUCUM RS,RS,21/10/2020


In [ ]:
#One way to do it
df.at[837,'DATA'] = '30/04/2020'
df.at[1503,'DATA'] = '16/10/2020'

In [ ]:
df_date = df['DATA DE ENTREGA'].str.contains('[0-9]{2}.[0-9]{2}.[0-9]{4}')
df_date = df_date.loc[df_date == False]
df_date

Series([], Name: DATA DE ENTREGA, dtype: bool)

In [ ]:
df_date = pd.to_datetime(df['DATA DE ENTREGA'], errors='coerce')
error = df_date.loc[df_date.isnull()]
error

Series([], Name: DATA DE ENTREGA, dtype: datetime64[ns])

In [ ]:
df.loc[[837]]

,DATA,FORNECEDOR,DESTINO,ESTADO/MUNICIPIO,TIPO,QUANTIDADE,VALOR,DESTINATARIO,UF,DATA DE ENTREGA
837,30/04/2020,LEISTUNG,PARA,ESTADO,UTI,60,3600000.0,Secretaria Estadual de Saúde,PA,01/05/2020


In [ ]:
df.at[837,'DATA DE ENTREGA'] = '01/05/2020'

In [ ]:
#Create a dataframe that excludes our broken data for calculating delay
df_delay = df.copy()
df_delay = df_delay.drop([300],axis=0)
df_delay['DATA DE ENTREGA'] = pd.to_datetime(df_delay['DATA DE ENTREGA'], format="%d/%m/%Y")
df_delay['DATA'] = pd.to_datetime(df_delay['DATA'], format="%d/%m/%Y")
df_delay['DELAY'] = df_delay['DATA DE ENTREGA'] - df_delay['DATA']
sorted = df_delay.sort_values(by='DELAY')
sorted.head()

,DATA,FORNECEDOR,DESTINO,ESTADO/MUNICIPIO,TIPO,QUANTIDADE,VALOR,DESTINATARIO,UF,DATA DE ENTREGA,DELAY
829,2020-06-02,MAGNAMED,RORAIMA,ESTADO,TRANSPORTE,15,724350.0,Secretaria Estadual de Saúde,RR,2020-04-01,-62 days
828,2020-06-02,VYAIRE,RORAIMA,ESTADO,UTI,15,900000.0,Secretaria Estadual de Saúde,RR,2020-04-01,-62 days
0,2020-04-19,VYAIRE,RIO DE JANEIRO,ESTADO,UTI,40,2400000.0,ALMOXARIFADO ESTADUAL DE MEDICAMENTOS - RIO DE...,RJ,2020-04-19,0 days
1299,2020-06-25,KTK,SÃO PAULO,ESTADO,UTI,10,600000.0,Secretaria Estadual de Saúde,SP,2020-06-25,0 days
29,2020-06-08,KTK,RIO DE JANEIRO,ESTADO,TRANSPORTE,50,1000000.0,ALMOXARIFADO ESTADUAL DE MEDICAMENTOS - RIO DE...,RJ,2020-06-08,0 days


In [38]:
#Exclude the shitty columns 829 and 828
df_delay = df_delay.drop([828,829], axis=0)
df_delay.head()

,DATA,FORNECEDOR,DESTINO,ESTADO/MUNICIPIO,TIPO,QUANTIDADE,VALOR,DESTINATARIO,UF,DATA DE ENTREGA,DELAY
0,2020-04-19,VYAIRE,RIO DE JANEIRO,ESTADO,UTI,40,2400000.0,ALMOXARIFADO ESTADUAL DE MEDICAMENTOS - RIO DE...,RJ,2020-04-19,0 days
1,2020-04-22,VYAIRE,RIO DE JANEIRO,ESTADO,UTI,20,1200000.0,LABORATÓRIO QUÍMICO FARMACÊUTICO DE AERONÁUTICA,RJ,2020-04-23,1 days
2,2020-05-08,MAGNAMED,RIO DE JANEIRO,ESTADO,UTI,25,1500000.0,ALMOXARIFADO ESTADUAL DE MEDICAMENTOS - RIO DE...,RJ,2020-05-09,1 days
3,2020-05-08,KTK,RIO DE JANEIRO,ESTADO,UTI,15,900000.0,ALMOXARIFADO ESTADUAL DE MEDICAMENTOS - RIO DE...,RJ,2020-05-09,1 days
4,2020-05-09,LEISTUNG,RIO DE JANEIRO,ESTADO,UTI,6,360000.0,ALMOXARIFADO ESTADUAL DE MEDICAMENTOS - RIO DE...,RJ,2020-05-09,0 days


In [39]:
df_delay.describe()

,QUANTIDADE,VALOR,DELAY
count,1644.000000,1.644000e+03,1644
mean,7.272506,3.797893e+05,5 days 08:00:52.554744525
std,13.743497,7.410215e+05,3 days 11:22:24.755114401
min,1.000000,2.000000e+04,0 days 00:00:00
25%,2.000000,6.000000e+04,3 days 00:00:00
50%,3.000000,1.448700e+05,5 days 00:00:00
75%,7.000000,3.600000e+05,7 days 00:00:00
max,300.000000,1.448700e+07,32 days 00:00:00


In [ ]:
supp_by_state = df.groupby(by=['FORNECEDOR'])['QUANTIDADE'].sum()
supp_by_state

FORNECEDOR
KTK                                     1539
KTK/VAYIRE                                 5
L C DADDE COM AT INST MAT MEDICO EPP      20
LEISTUNG                                1096
LEISTUNG/VYAIRE                           20
Leistung                                 100
MAGNAMED                                4726
MAGNAMED-REQUISIÇÃO                       30
MAGNAMED/REQUISIÇÃO                       30
SUZANO/Resmed                             10
SUZANO/Zhongxun Medical                   10
UTI                                        3
VYAIRE                                  3372
VYAIRE LTV 1200 USA                      200
VYAIRE LTV 2200 USA                      350
WEG                                      500
Name: QUANTIDADE, dtype: int64

In [ ]:
mapper = {"KTK/VAYIRE":"KTK","LEISTUNG/VYAIRE":"LEISTUNG","Leistung":"LEISTUNG","MAGNAMED-REQUISIÇÃO":"MAGNAMED",
          "MAGNAMED/REQUISIÇÃO":"MAGNAMED","SUZANO/RESMED":"SUZANO","SUZANO/Resmed":"SUZANO","SUZANO/Zhongxun Medical":"SUZANO",
          "VYAIRE LTV 1200 USA":"VYAIRE", "VYAIRE LTV 2200 USA":"VYAIRE"}

df_fornecedor = df.copy()
df_fornecedor['FORNECEDOR'] = df_fornecedor['FORNECEDOR'].replace(mapper)
df_fornecedor.tail()

,DATA,FORNECEDOR,DESTINO,ESTADO/MUNICIPIO,TIPO,QUANTIDADE,VALOR,DESTINATARIO,UF,DATA DE ENTREGA
1642,24/09/2020,VYAIRE,PARANA,MUNICIPIO,TRANSPORTE USA,1,63125.97,SMS de GUAIRA,PR,28/09/2020
1643,08/10/2020,VYAIRE,PARANA,ESTADO,TRANSPORTE USA,20,1262519.40,5o Batalhao de Suprimento,PR,17/10/2020
1644,08/10/2020,MAGNAMED,PARANA,ESTADO,UTI,4,240000.00,5o Batalhao de Suprimento,PR,17/10/2020
1645,16/10/2020,MAGNAMED,PARANA,MUNICIPIO,UTI,1,60000.00,SMS de WENCESLAU BRAZ,PR,21/10/2020
1646,07/08/2020,MAGNAMED,LIBANO,-,TRANSPORTE,300,14487000.00,MISSÃO FAB (Doação Destino ao Libano).,-,08/08/2020


In [ ]:
equip_by_supp = df_fornecedor.groupby(by=['FORNECEDOR'])['QUANTIDADE'].sum()
equip_by_supp

FORNECEDOR
KTK                                     1544
L C DADDE COM AT INST MAT MEDICO EPP      20
LEISTUNG                                1216
MAGNAMED                                4786
SUZANO                                    20
UTI                                        3
VYAIRE                                  3922
WEG                                      500
Name: QUANTIDADE, dtype: int64